In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin


In [ ]:
### Standard Imports
import numpy as np
import re
import sys
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils

In [ ]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'

### General Parameters
random_seed = 2022
model_folder = '../../models/autoencoder/lstm/v1'

### Model Parameters
window_len = 15
batch_size = 64
enc_dim, dec_dim = 256, 256
learn_rate = 0.001
epochs = 80
dropout = 0.05
recurrent_dropout = 0.05

In [ ]:
### Load Data
corpus = utils.load_corpus()

In [ ]:
### Pre-Processing Text
words = utils.preprocess_text(corpus, fun_list = [utils.to_lower, utils.decontraction, utils.remove_punct], keep = '\<|\>')
words = re.sub('\n',' \n ', words)
words = re.split(' +', words) #Tokenising

word_count = Counter(words) # Assumes end_token is already in the corpus
word_count[start_token] = 0
word_count[pad_token] = 0

#Reference Dictionaries to convert one-hot index to string and vice versa
index_to_vocab = {i: k for i, k in enumerate(word_count.keys())}
vocab_to_index = {k: i for i, k in enumerate(word_count.keys())}

songs = ' '.join(words)
songs = songs.split(' \n \n <eos> \n \n ')
songs = [song.split(' ') for song in songs]
songs = [[pad_token]*(window_len-1) + [start_token] + song + [end_token] + [pad_token]*(window_len-1) for song in songs]
songs_token_ind = [[vocab_to_index.get(x) for x in song] for song in songs]

In [ ]:
### Creating Dataset
x_encoder = []
x_decoder = []
y = []
vocab_size = len(word_count)

for song in songs_token_ind:
  for i in range(len(song)-window_len):
    x_encoder.append(song[(i+1):(i+window_len+1)])
    x_decoder.append(song[i:(i+window_len)])
    y.append(song[i+window_len])

In [ ]:
rand_int = np.random.randint(0, len(x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in x_decoder[rand_int]])
print(index_to_vocab.get(y[rand_int]))

['hand', '\n', 'pick', 'yourself', 'up', 'pick', 'yourself', 'up', 'get', 'ready', '\n', 'we', 'will', 'go', 'make']
['a', 'hand', '\n', 'pick', 'yourself', 'up', 'pick', 'yourself', 'up', 'get', 'ready', '\n', 'we', 'will', 'go']
make


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((x_encoder, x_decoder), y))
dataset = dataset.shuffle(buffer_size = 10000, seed = random_seed)
dataset = dataset.map(lambda x, y: ((tf.one_hot(x[0], depth = vocab_size), tf.one_hot(x[1], depth = vocab_size)),
                                 tf.one_hot(y, depth = vocab_size)))
train_dataset = dataset.take(int((1-val_split)*len(dataset)))
val_dataset = dataset.skip(int((1-val_split)*len(dataset)))

train_dataset_final = train_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_final = val_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(window_len,vocab_size), name = 'encoder_input')

# Return state in addition to output
encoder_output, encoder_hidden_state, encoder_cell_state = layers.LSTM(enc_dim,
                                                                       dropout = dropout, recurrent_dropout = recurrent_dropout,
                                                                       return_state=True, name = "encoder_lstm")(encoder_input)

# Decoder
decoder_input = layers.Input(shape=(window_len,vocab_size), name = 'decoder_input')

# Pass the encoder state to a new LSTM, as initial state
decoder_output = layers.LSTM(dec_dim,
                             dropout = dropout, recurrent_dropout = recurrent_dropout,
                             name="decoder_lstm")(decoder_input, initial_state=[encoder_hidden_state, encoder_cell_state])
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(decoder_output)

model = tf.keras.Model((encoder_input, decoder_input), output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 15, 1049)]   0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, 15, 1049)]   0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 256),        1337344     ['encoder_input[0][0]']          
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate),
              metrics = ['accuracy'])

In [ ]:
### Callbacks
callback_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

callback_mc = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = model.fit(x = train_dataset_final, validation_data = val_dataset_final, epochs = epochs, callbacks = [callback_es, callback_mc])

Epoch 1/100
226/226 [==============================] - 33s 119ms/step - loss: 5.2904
Epoch 2/100
226/226 [==============================] - 27s 117ms/step - loss: 4.8961
Epoch 3/100
226/226 [==============================] - 26s 117ms/step - loss: 4.6287
Epoch 4/100
226/226 [==============================] - 28s 123ms/step - loss: 3.8758
Epoch 5/100
226/226 [==============================] - 26s 116ms/step - loss: 3.2250
Epoch 6/100
226/226 [==============================] - 26s 115ms/step - loss: 2.8258
Epoch 7/100
226/226 [==============================] - 30s 132ms/step - loss: 2.8006
Epoch 8/100
226/226 [==============================] - 26s 115ms/step - loss: 2.6564
Epoch 9/100
226/226 [==============================] - 28s 123ms/step - loss: 2.3794
Epoch 10/100
226/226 [==============================] - 26s 116ms/step - loss: 2.1830
Epoch 11/100
226/226 [==============================] - 26s 117ms/step - loss: 2.0493
Epoch 12/100
226/226 [==============================] - 29s 127

In [ ]:
model.save_weights(model_folder)

In [ ]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0, random_seed = 2022):
    # Converting our start string to numbers (vectorizing).
    input_indices = [vocab_to_index.get(s) for i, s in enumerate(start_string) if i < window_len-1]
    input_indices = [vocab_to_index.get(pad_token)]*(window_len - len(input_indices)-1) + [vocab_to_index.get(start_token)] + input_indices

    input_oh = tf.one_hot(input_indices, depth = vocab_size)
    x = tf.expand_dims(input_oh, 0)

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1.
    model.reset_states()
    for word_index in range(num_generate):
        prediction = model.predict([x,x])

        # Using a categorical distribution to predict the character returned by the model.
        prediction = prediction / temperature
        predicted_id = tf.random.categorical(prediction, num_samples=1, seed = random_seed)[-1,0]
        predicted_oh = tf.one_hot(predicted_id, depth = vocab_size)

        # We pass the series of previous words (up to window length) as the next input to the model
        # along with the previous hidden state.
        input_index = tf.expand_dims([predicted_oh], 0)
        x = tf.concat([x[:,1:,:],input_index], 1)
        
        pred_word = index_to_vocab[predicted_id.numpy()]
        text_generated.append(pred_word)
        if pred_word == end_token:
          break
    
    return (' '.join(start_string) + ' ' + ' '.join(text_generated)), text_generated

In [ ]:
result_str, result = generate_text(model, start_string=['<verse>','\n','step','by','step'], num_generate=50, temperature=1.0)

In [ ]:
print(result_str)

<verse> 
 step by step progressing day now moving her savour days coming night pure soldiers what’s grows independence moving bombay homely here soar current romance island hollywood seen difference five aspire mountain minute looking price help leap london waiting must single wall adore favourite light these learnt grateful <cls> grateful truly moments broadway walked
